https://deepinfra.com/blog/function-calling-feature

Pros:

User can supply any kind of functions to the model (aka. Assistant). The Assistant will be useful, as long as the descriptions of the functions are well-written.
Cons:

User (most likely Developer) needs to write some code, or have a way to execute the function with the parameters that the model decided to call.
During the interaction the LLM model might ask clarifying questions in order to execute functions with the right arguments.

Use cases
Here are just a few example use cases that can benefit from Function Calling capability of LLMs:

Real-time data processor: you can provide a function that accesses the real-time data and does some actions
Math Problem Solver: it can be used to solve complex math problems (e.g. by providing a function that talks to WolframAlpha or other third-party tools)
AI Virtual Assistants: you can enhance your AI assistant with various functionalities that the LLM model can call and respond back meaningfully to the end user
Now, let's consider a concrete example.

Example with code
Let's consider this example:

User has a function get_current_weather(location)
User wants to find out the city with the hottest weather.
Here is the user prompt

In [1]:
messages = [
    {
        "role": "user",
        "content": "Which city has the hottest weather today: San Francisco, Tokyo, or Paris?"
    }
]

In [2]:
tools = [{
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description":
                        "The city and state, e.g. San Francisco, CA"
                }
            },
            "required": ["location"]
        },
    }
}]

In [3]:
import openai

client = openai.OpenAI(
    base_url="https://api.deepinfra.com/v1/openai",
    api_key="<Your-DeepInfra-API-Key>",
)

response = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    messages=messages,
    tools=tools,
    tool_choice="auto",
)

tool_calls = response.choices[0].message.tool_calls
for tool_call in tool_calls:
    print(tool_call.model_dump())

ModuleNotFoundError: No module named 'openai'

In [4]:
import json
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location):
    """Get the current weather in a given location"""
    print("Calling get_current_weather client side.")
    if "tokyo" in location.lower():
        return json.dumps({
            "location": "Tokyo",
            "temperature": "75"
        })
    elif "san francisco" in location.lower():
        return json.dumps({
            "location": "San Francisco",
            "temperature": "60"
        })
    elif "paris" in location.lower():
        return json.dumps({
            "location": "Paris",
            "temperature": "70"
        })
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [ ]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    if function_name == "get_current_weather":
        function_args = json.loads(tool_call.function.arguments)
        function_response = get_current_weather(
            location=function_args.get("location")
        )

    # extend conversation with function response
    messages.append({
        "tool_call_id": tool_call.id,
        "role": "tool",
        "content": function_response,
    })


# get a new response from the model where it can see the function responses
second_response = client.chat.completions.create(
    model="mistralai/Mistral-7B-Instruct-v0.1",
    messages=messages,
)

print(second_response.choices[0].message.content)